## CSE595 Homework 1 : Logistic Regression Implementation

In [3]:
import pandas

### Part 1 : Representing Text Data

In [ ]:
# tokenizes purely by using whitespace
def tokenize (sentence : str):
    tokens = sentence.split()
    return tokens

In [ ]:
punct = set([",", ".", "!", "?", "#", "$", "%", ";", ":", "(", ")"])

def better_tokenize(sentence : str):

#### Testing Tokenization

In [9]:
df = pandas.read_csv("train.csv")
print((df['generation'][1000]))

Sure! Here's a recipe for a delicious and healthier "ice cream" cake that combines the rich flavors of mocha and crunchy yogurt:

Ingredients:

For the crust:

* 1 1/2 cups graham cracker crumbs
* 1/4 cup granulated sugar
* 1/2 cup unsalted butter, melted

For the ice cream layers:

* 3 cups plain Greek yogurt
* 1/2 cup strong brewed coffee
* 1/2 cup granulated sugar
* 1 tablespoon unsweetened cocoa powder
* 1 teaspoon vanilla extract
* 1/4 teaspoon salt

For the chocolate ganache:

* 1/2 cup semisweet chocolate chips
* 1/4 cup heavy cream

For the whipped cream topping:

* 1 cup heavy cream
* 1 tablespoon granulated sugar

Instructions:

1. Preheat your oven to 350°F (180°C). 

2. In a medium-sized bowl, mix together the graham cracker crumbs and sugar. 

Pour in the melted butter and stir until well combined. 

Press the mixture into the bottom and up the sides of a 9-inch springform pan. Bake for 10-12 minutes or until lightly golden brown. Let cool completely. 3. In a large mixing 